In [2]:
# Import the necessary packages to perform the data analysis

# packages to process and visualize the data
import pandas as pd
import numpy as np
import seaborn as sns
import scipy
import matplotlib.pyplot as plt

# import othe useful packages
import os

# packages to improve visual description and analysis
from IPython.core import display as ICD
pd.set_option('display.max_columns', 100)

# Sociological Research

**1)** Map the tax heavens jurisdiction in terms of the number of entities they have registered 

**2)** Map countries in the world according to how much they appear in the Panama Papers in terms of the entities

**3)** Explore the GINI coefficient which is a measure of inequality: what is the correlation between counts and gini?

**4)** Explore the HDI coefficient which is a measure of development: what is the correlation between counts and HDI?

**5)** Explore the indices of informal sector, organized crime coefficient which is a measure of inequality: what is the correlation between counts and these indicators?

# Economical Research

**1)** Find how many companies in Fortune 2000 are involved in Panama Papers, present the top rank in the result.

**2)** Timeline of number of leaked papers over the years (graph). Try to extrapolate a tendecy in this phenomena.

**3)** Amount of lost capital for a country due to tax evasion.
